# Building the bot

In this notebook, we'll create a bot that will be used as the interface to analyze the enriched events from the previous notebook. The bot will allow users to:

- Ask what's trending on Bluesky
- Search for posts based on topics

## Setting Up the Environment
In this section, we'll set up the environment for our bot. We'll import the necessary libraries and create the HTTP client that we'll use to interact with the Bluesky API.


In [1]:
%use coroutines

### Defining Authentication Data Models
We need to define data classes to represent the response from the Bluesky authentication API. These models will be used to deserialize the JSON response from the API.


In [2]:
import kotlinx.serialization.SerialName
import kotlinx.serialization.Serializable

@Serializable
data class LoginResponse(
    @SerialName("accessJwt") val accessJwt: String,
    @SerialName("refreshJwt") val refreshJwt: String,
    @SerialName("handle") val handle: String,
    @SerialName("did") val did: String,
    @SerialName("didDoc") val didDoc: DidDoc?,
    @SerialName("email") val email: String?,
    @SerialName("emailConfirmed") val emailConfirmed: Boolean?,
    @SerialName("emailAuthFactor") val emailAuthFactor: Boolean?,
    @SerialName("active") val active: Boolean,
    @SerialName("status") val status: String? = null
)

@Serializable
data class DidDoc(
    @SerialName("id") val id: String?
)

### Implementing Authentication
Now we'll implement the function to authenticate with the Bluesky API. This function will send a POST request to the API with our username and password, and return an access token that we can use for subsequent requests.


In [3]:
val API_URL = "https://bsky.social/xrpc"
val USERNAME = "devbubble.bsky.social"
val PASSWORD = System.getenv("DEVBUBBLE_TOKEN")

In [4]:
import dev.raphaeldelio.*
import io.ktor.client.call.body
import io.ktor.client.request.HttpRequestBuilder
import io.ktor.client.request.headers
import io.ktor.client.request.post
import io.ktor.client.request.setBody
import io.ktor.http.ContentType
import io.ktor.http.HeadersBuilder
import io.ktor.http.HttpStatusCode
import io.ktor.http.contentType

suspend fun getAccessToken(): String {
    val response = httpClient.post("$API_URL/com.atproto.server.createSession") {
        contentType(ContentType.Application.Json)
        setBody(
            mapOf(
                "identifier" to USERNAME,
                "password" to PASSWORD
            )
        )
    }

    return if (response.status == HttpStatusCode.OK) {
        val result: LoginResponse = response.body()
        jedisPooled.set("mainDid", result.did)
        println("✅ Login successful. DID: ${result.did}")
        result.accessJwt
    } else {
        println("⚠️ Authentication failed: ${response.status}")
        ""
    }
}

### Getting the Access Token
Now we'll call the `getAccessToken` function to get an access token from the Bluesky API. We'll store this token in a variable that we can use for subsequent requests.


In [5]:
var blueskyToken: String
runBlocking {
    blueskyToken = getAccessToken()
}

✅ Login successful. DID: did:plc:qdwb7czl4gdbu5go25dza3vo


## Searching for Posts
In this section, we'll implement the functionality to search for posts on Bluesky. We'll define data models to represent the search response and posts, and then implement a function to search for posts.

### Defining Post Search Models
First, we need to define data classes to represent the response from the Bluesky search API. These models will be used to deserialize the JSON response from the API.


In [6]:
@Serializable
data class SearchResponse(
    @SerialName("cursor") val cursor: String? = null,
    @SerialName("hitsTotal") val hitsTotal: Int? = null,
    @SerialName("posts") val posts: List<Post>
)

@Serializable
data class Post(
    @SerialName("uri") val uri: String,
    @SerialName("cid") val cid: String,
    @SerialName("author") val author: Author,
    @SerialName("indexedAt") val indexedAt: String,
    @SerialName("record") val record: Record?,
    @SerialName("replyCount") val replyCount: Int? = null,
    @SerialName("repostCount") val repostCount: Int? = null,
    @SerialName("likeCount") val likeCount: Int? = null,
    @SerialName("quoteCount") val quoteCount: Int? = null,

    )

@Serializable
data class Author(
    @SerialName("did") val did: String,
    @SerialName("handle") val handle: String,
    @SerialName("displayName") val displayName: String? = null,
    @SerialName("avatar") val avatar: String? = null
)

@Serializable
data class Record(
    @SerialName("text") val text: String? = null,
    @SerialName("embed") val embed: Embed? = null,
    @SerialName("createdAt") val createdAt: String
)

@Serializable
data class Embed(
    @SerialName("images") val images: List<Image>? = null
)

@Serializable
data class Image(
    @SerialName("thumb") val thumb: String? = null, // Nullable to handle missing values
    @SerialName("fullsize") val fullsize: String? = null,
    @SerialName("alt") val alt: String? = null // Alt text is also optional
)

### Implementing Post Search
Now we'll implement the function to search for posts on Bluesky. This function will send a GET request to the API with our search parameters, and return a list of posts that match our search criteria.

The function uses pagination to retrieve all posts that match the search criteria, even if there are more than can be returned in a single response.


In [7]:
import io.ktor.client.*
import io.ktor.client.request.*
import io.ktor.client.statement.*
import io.ktor.client.call.*
import io.ktor.http.*

import java.time.Instant
import java.time.temporal.ChronoUnit

suspend fun searchPosts(sinceTime: String, term: String): List<Post> {
    val allPosts = mutableListOf<Post>()
    var cursor: String? = null

    println("🔍 Searching posts with tag: $term since: $sinceTime")
    do {
        val response: HttpResponse = httpClient.get("$API_URL/app.bsky.feed.searchPosts") {
            headers {
                append("Authorization", "Bearer $blueskyToken")
            }
            parameter("q", term)
            parameter("sort", "latest")
            parameter("limit", 100)
            parameter("since", sinceTime)
            if (cursor != null) {
                parameter("cursor", cursor)
            }
        }

        if (response.status == HttpStatusCode.OK) {
            val result: SearchResponse = response.body()
            val posts = result.posts
            println("✅ Retrieved ${posts.size} posts. Total so far: ${allPosts.size + posts.size}.")
            allPosts.addAll(posts)
            cursor = result.cursor
        } else {
            println("⚠️ Failed to fetch posts. Status: ${response.status}")
            println(response.bodyAsText())
            break
        }
    } while (cursor != null)

    println("🎉 Finished fetching posts. Total retrieved: ${allPosts.size}.")
    return allPosts
}

### Testing Post Search
Let's test our post search function by searching for posts that mention our bot's handle. We'll search for posts from the last 15 hours, and for each post, we'll process the text to remove our bot's handle and then process the user's request.


In [8]:
val sinceTime = Instant.now().minus(15, ChronoUnit.HOURS).toString()
runBlocking {
    val posts = searchPosts(sinceTime, "@devbubble.bsky.social")
    posts.forEach { post ->
        post.record?.text?.replace("@devbubble.bsky.social", "")?.trim()!!.let { cleanedPost ->
            println(processUserRequest(cleanedPost, multiHandler))
        }
    }
}

🔍 Searching posts with tag: @devbubble.bsky.social since: 2025-05-22T20:47:18.474881Z
✅ Retrieved 1 posts. Total so far: 1.
🎉 Finished fetching posts. Total retrieved: 1.
What's being said about ChatGPT
summarization
0.8501898050308228
0.8

[summarization]
[ChatGPT, AI Chatbots, OpenAI, Generative Pre-trained Transformer, Natural Language Processing]
[[Vercel Releases v0 AI Model for Web Application Development, Compatible with OpenAI API Vercel, the company behind the vibe coding platform for web application development, v0, is now releasing an artificial intelligence (AI) model. Announced on Thursda...

| Details | Interest | Feed |, Wow Claude 4.0 😍 , we are in a different world !, OpenAI vs. R/Whereintheworld Article URL: https://www.whereisthisphoto.com/blog/openai-model-image-analysis Comments URL: https://news.ycombinator.com/item?id=44071449 Points: 2 # Comments: 1 

| Details | Interest | Feed |, did you seriously need chatgpt to make this image? the illustration with text on 

## Creating Posts
In this section, we'll implement the functionality to create posts on Bluesky. We'll define data models to represent the post request and then implement a function to create posts.

### Defining Post Creation Models
First, we need to define data classes to represent the request we'll send to the Bluesky API to create a post. These models will be used to serialize our request into JSON.


In [9]:
@Serializable
data class ReplyRef(
    val root: PostRef,
    val parent: PostRef
)

@Serializable
data class PostRef(
    val cid: String,
    val uri: String
)

@Serializable
data class PostRecord(
    val `$type`: String = "app.bsky.feed.post",
    val text: String,
    val createdAt: String,
    val reply: ReplyRef? = null
)

@Serializable
data class PostRequest(
    val repo: String,
    val collection: String,
    val record: PostRecord
)


### Implementing Post Creation
Now we'll implement the function to create posts on Bluesky. This function will send a POST request to the API with our post content, and return a boolean indicating whether the post was created successfully.

The function can create both standalone posts and replies to existing posts. If `replyToUri` and `replyToCid` are provided, the post will be created as a reply to the specified post.


In [10]:
suspend fun createPost(
    text: String,
    replyToUri: String? = null,
    replyToCid: String? = null
): Boolean {
    val replyRef = if (replyToUri != null && replyToCid != null) {
        ReplyRef(
            root = PostRef(uri = replyToUri, cid = replyToCid),
            parent = PostRef(uri = replyToUri, cid = replyToCid)
        )
    } else {
        null
    }

    val record = PostRecord(
        text = text,
        createdAt = Instant.now().toString(),
        reply = replyRef
    )

    val response: HttpResponse = httpClient.post("$API_URL/com.atproto.repo.createRecord") {
        headers {
            append("Authorization", "Bearer $blueskyToken")
            contentType(ContentType.Application.Json)
        }
        setBody(
            PostRequest(
                repo = "did:plc:qdwb7czl4gdbu5go25dza3vo",
                collection = "app.bsky.feed.post",
                record = record
            )
        )
    }

    return if (response.status == HttpStatusCode.OK || response.status == HttpStatusCode.Accepted) {
        println("✅ Post created${if (replyRef != null) " (as reply)" else ""}!")
        true
    } else {
        println("❌ Failed to create post: ${response.status}")
        println(response.bodyAsText())
        false
    }
}

In [11]:
runBlocking {
    createPost("test")
}

✅ Post created!


true

### Putting It All Together

In [12]:
fun splitIntoChunks(text: String, maxLength: Int = 300): List<String> {
    val words = text.split(Regex("\\s+"))
    val chunks = mutableListOf<String>()
    var current = StringBuilder()

    for (word in words) {
        if (current.length + word.length + 1 > maxLength) {
            chunks.add(current.toString().trim())
            current = StringBuilder()
        }
        current.append(word).append(' ')
    }

    if (current.isNotEmpty()) {
        chunks.add(current.toString().trim())
    }

    return chunks
}

In [13]:
val embeddingModel = getEmbeddingModelForRouting()
val redisVectorStore = getSemanticCacheRedisVectorStore(embeddingModel)

In [15]:
import dev.raphaeldelio.*

val sinceTime = Instant.now().minus(15, ChronoUnit.HOURS).toString()
runBlocking {
    val posts = searchPosts(sinceTime, "@devbubble.bsky.social")
    posts.forEach { post ->
        post.record?.text?.replace("@devbubble.bsky.social", "")?.trim()!!.let { cleanedPost ->
            val handle = post.author.handle
            var lastUri = post.uri
            var lastCid = post.cid

            val cachedResponse = getFromCache(redisVectorStore, cleanedPost)
            val response = if (cachedResponse.isNotBlank()) {
                cachedResponse
            } else {
                val response = processUserRequest(cleanedPost, multiHandler)
                insertIntoCache(redisVectorStore, cleanedPost, response)
                response
            }

            val finalResponse = "@$handle $response"
            val chunks = splitIntoChunks(finalResponse)

            chunks.forEach { chunk ->
                createPost(
                    chunk,
                    lastUri,
                    lastCid
                )
                lastUri = post.uri
                lastCid = post.cid
            }
        }
    }
}

🔍 Searching posts with tag: @devbubble.bsky.social since: 2025-05-22T20:49:19.030344Z
✅ Retrieved 1 posts. Total so far: 1.
🎉 Finished fetching posts. Total retrieved: 1.
What's being said about OpenAI
summarization
0.8119206428527832
0.8

[summarization]
[OpenAI, AI Ethics, Technology Trends]
[[Vercel Releases v0 AI Model for Web Application Development, Compatible with OpenAI API Vercel, the company behind the vibe coding platform for web application development, v0, is now releasing an artificial intelligence (AI) model. Announced on Thursda...

| Details | Interest | Feed |, Wow Claude 4.0 😍 , we are in a different world !, OpenAI vs. R/Whereintheworld Article URL: https://www.whereisthisphoto.com/blog/openai-model-image-analysis Comments URL: https://news.ycombinator.com/item?id=44071449 Points: 2 # Comments: 1 

| Details | Interest | Feed |, did you seriously need chatgpt to make this image? the illustration with text on it? that's the level of laziness on display?, Arrivano Cla